In [ ]:
import numpy as np
import xarray as xr
import xskillscore as xs

In [ ]:
FORECAST = "***BASEDIR***/runs/infer/outputs/2021-09-07/14-28-19/normal_normal_monthly.nc"
OBS = "***BASEDIR***/last-validation-files-from-organizers/forecast-like-observations_2020_biweekly_terciled.nc"
REFERENCE = "***BASEDIR***//last-validation-files-from-organizers/ML_prediction_2020.nc"

reference = xr.open_dataset(REFERENCE)

In [ ]:
FORECAST = "***BASEDIR***/ml-output/2021-08-30/normal_normal_monthly-without-validtime.nc"
OBS = "***BASEDIR***2/from-template-lfs/hindcast-like_observations_2000-2019_biweekly_terciled.nc"

reference_t2m = xr.open_dataset('***BASEDIR***/training-output-benchmark/t2m.nc')
reference_tp = xr.open_dataset('***BASEDIR***/training-output-benchmark/tp.nc')
reference = xr.merge([reference_t2m, reference_tp])

In [ ]:
reference

In [ ]:
def make_climatology():
    clim = xr.DataArray([1/3, 1/3, 1/3], dims='category', coords={'category':['below normal', 'near normal', 'above normal']}).to_dataset(name='tp')
    clim['t2m'] = clim['tp']
    
    return clim

In [ ]:
def compute_arlan_rpss(forecast_rps, clim_rps):
    """
    Compute RPSS according to Arlan's definition
    """

    weights = np.cos(np.deg2rad(np.abs(clim_rps.latitude)))

    latitude_longitude_sum_ML = (forecast_rps) * weights
    latitude_longitude_sum_clim = (clim_rps) * weights

    return 1 - latitude_longitude_sum_ML/latitude_longitude_sum_clim

In [ ]:
forecast = xr.open_dataset(FORECAST)

In [ ]:
obs = xr.open_dataset(OBS)

In [ ]:
reference = reference.sel(forecast_time=forecast.forecast_time)

In [ ]:
rps_ml = xs.rps(obs, forecast, category_edges=None, dim='forecast_time', input_distributions='p')         
rps_clim = xs.rps(obs, make_climatology(), category_edges=None, dim='forecast_time', input_distributions='p')
rps_ref = xs.rps(obs, reference, category_edges=None, dim='forecast_time', input_distributions='p')

In [ ]:
rpss_ours_clim = compute_arlan_rpss(rps_ml, rps_clim)

In [ ]:
rpss_ours_reference = compute_arlan_rpss(rps_ml, rps_ref)

In [ ]:
rpss_ours_clim

In [ ]:
1.0 - (rps_ml.sum(dim=['latitude', 'longitude']) / rps_ref.sum(dim=['latitude', 'longitude']))

In [ ]:
rpss_ours_clim.t2m.sel(latitude=slice(90,-60)).plot(size=4, aspect=2, col='lead_time')

In [ ]:
rpss_ours_clim.tp.sel(latitude=slice(90,-60)).plot(size=4, aspect=2, col='lead_time')

In [ ]:
rpss_ours_clim.isel(lead_time=0).tp.plot()

In [ ]:
rpss_ours_reference.isel(lead_time=0).t2m.plot()

In [ ]:
rps_ref.isel(lead_time=0).t2m.plot()

In [ ]:
rpss.isel(lead_time=0).t2m.plot()

In [ ]:
reference.t2m.isel(lead_time=0, category=0, forecast_time=0)

In [ ]:
rps_ref.isel(lead_time=0).t2m.plot()

In [ ]:
rps_ml.mean()

In [ ]:
rps_ml.t2m.isel(lead_time=1).plot()

In [ ]:
rps_clim.t2m.isel(lead_time=1).plot()

In [ ]:
1.0 - (rps_ml.sum(dim=['latitude', 'longitude']) / rps_clim.sum(dim=['latitude', 'longitude']))

In [ ]:
rps_ml.mean(dim=['latitude', 'longitude'])

In [ ]:
rps_clim.mean(dim=['latitude', 'longitude'])